In [20]:
import os
from glob import glob
import pandas as pd
import numpy as np
import cv2
from model import *
from util import *

In [21]:
n_epochs = 10000
learning_rate_val = 0.002
weight_decay_rate =  0.00001
momentum = 0.9
batch_size = 100
lambda_recon = 0.999
lambda_adv = 0.001

In [22]:
trainset_path = '../data/paris_trainset.pickle'
testset_path  = '../data/paris_testset.pickle'
# dataset_path = '/media/storage3/Study/data/Paris/'
dataset_path = '/home/lab/PycharmProjects/Inpainting/DataSet/Paris/'
model_path = '../models/Paris2/'
result_path= '../results/Paris2/'
pretrained_model_path = None# '../models/Paris/model-0'

In [23]:
if not os.path.exists(model_path):
    os.makedirs( model_path )

In [24]:
if not os.path.exists(result_path):
    os.makedirs( result_path )

In [33]:
if not os.path.exists( trainset_path ) or not os.path.exists( testset_path ):
    print 'coming...'
    trainset_dir = os.path.join( dataset_path, 'paris_train_original' )
    testset_dir = os.path.join( dataset_path, 'paris_eval_gt' )
    trainset = pd.DataFrame({'image_path': map(lambda x: os.path.join( trainset_dir, x ), os.listdir(trainset_dir))})
    print trainset
    testset = pd.DataFrame({'image_path': map(lambda x: os.path.join( testset_dir, x ), os.listdir(testset_dir))})
    print testset
    
    trainset.to_pickle( trainset_path )
    testset.to_pickle( testset_path )
else:
    trainset = pd.read_pickle( trainset_path )
    testset = pd.read_pickle( testset_path )

In [34]:
testset.index = range(len(testset))
print testset.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
            85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
           dtype='int64')


In [35]:
testset = testset.iloc[np.random.permutation(len(testset))]
print testset

                                           image_path
73  /home/lab/PycharmProjects/Inpainting/DataSet/P...
94  /home/lab/PycharmProjects/Inpainting/DataSet/P...
93  /home/lab/PycharmProjects/Inpainting/DataSet/P...
10  /home/lab/PycharmProjects/Inpainting/DataSet/P...
98  /home/lab/PycharmProjects/Inpainting/DataSet/P...
26  /home/lab/PycharmProjects/Inpainting/DataSet/P...
88  /home/lab/PycharmProjects/Inpainting/DataSet/P...
2   /home/lab/PycharmProjects/Inpainting/DataSet/P...
77  /home/lab/PycharmProjects/Inpainting/DataSet/P...
57  /home/lab/PycharmProjects/Inpainting/DataSet/P...
89  /home/lab/PycharmProjects/Inpainting/DataSet/P...
95  /home/lab/PycharmProjects/Inpainting/DataSet/P...
6   /home/lab/PycharmProjects/Inpainting/DataSet/P...
11  /home/lab/PycharmProjects/Inpainting/DataSet/P...
1   /home/lab/PycharmProjects/Inpainting/DataSet/P...
8   /home/lab/PycharmProjects/Inpainting/DataSet/P...
25  /home/lab/PycharmProjects/Inpainting/DataSet/P...
40  /home/lab/PycharmProject

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))